In [1]:
import pandas as pd
from anytree import AnyNode
from anytree.exporter import DotExporter
from anytree.search import find
from anytree import PreOrderIter
from itertools import product
import json
import os
import numpy as np

print_mode = False

def find_node(root, node_to_find):
    """
    Traverse the tree to find the starting node by name.

    Parameters:
    root (Node): The root node of the tree
    node_to_find (string): The name of the node to find

    return:
    Node: The starting node if found, otherwise None
    """
    node = find(root, lambda node: node.name == node_to_find)
    if print_mode:
        if node:
            print(f"Node found: {node.name}")
        else:
            print("Node not found")

    return node

def generate_traces(node, calc_cost=False):
    """
    Recursively generates all possible traces from the given node.

    Parameters:
    node (Node): The current node in the tree.
    calc_cost (bool): Whether to calculate the cost of each trace.

    Returns:
    list: A list of all possible traces from the given node.
    list: A list of the cost of each trace.
    """
    # If the node has no children, it is a leaf node (ACT), end of a trace
    if not hasattr(node, 'children') or not node.children:
        if calc_cost:
            return [[node.name]], [node.costs]
        else:
            return [[node.name]], []

    traces = []
    costs = []

    if node.type == "OR":
        # OR node: Select one child at a time
        for child in node.children:
            child_traces, child_cost = generate_traces(child, calc_cost)
            for trace in child_traces:
                traces.append([node.name] + trace)
            if calc_cost:
                for cost in child_cost:
                    costs.append(cost)

    elif node.type == "SEQ" or node.type == "AND":
        # SEQ/AND node: Concatenate traces of all children in order
        child_traces = []
        child_costs = []
        for child in node.children:
            # Recursively generate traces and costs for each child
            child_traces_i, child_costs_i = generate_traces(child, calc_cost)
            child_traces.append(child_traces_i)
            child_costs.append(child_costs_i)

        # Generate all combinations of traces from children
        for combination in product(*child_traces):
            traces.append([node.name] + [step for trace in combination for step in trace])

        if calc_cost:
            # Sum the costs for each combination of child traces
            for combination in product(*child_costs):
                costs.append(np.sum([cost for cost in combination], axis=0))

    return traces, costs

def build_tree(json_node, parent=None):
    """
    Build the entire tree from the JSON object.

    Parameters:
    json_node (dict): The JSON object representing the tree
    parent (Node): The parent node

    Returns:
    Node: The root node of the tree
    """
    # Extract attributes from the JSON node, excluding 'name', 'type', and 'children'
    attributes = {k: v for k, v in json_node.items() if k not in ['name', 'type', 'children']}
    
    # Create a new node with the extracted attributes
    node = AnyNode(name=json_node['name'], type=json_node['type'], violation=False, parent=parent, **attributes)
    
    # Recursively build the tree for each child node
    for child in json_node.get('children', []):
        build_tree(child, node)
    
    return node

def annotate_tree(node, norm):
    """
    Annotates the tree by marking nodes that violate the given norm.

    Parameters:
    node (Node): The current node in the tree
    norm (dict): The norm

    - If the norm is of type 'P' (prohibited), a node violates it if its name is in norm['actions'].
    - If the norm is of type 'O' (obligatory), a node violates it if it is an action but not in norm['actions'].
    """
    # Recursively annotate each child node
    for child in node.children:
        annotate_tree(child, norm)

    # Check if the current node violates the norm
    if 'type' in norm:
        if norm['type'] == 'P':
            # Prohibited norm: node violates if its name is in norm['actions']
            node.violation = node.name in norm['actions']
        elif norm['type'] == 'O':
            # Obligatory norm: node violates if it is an action but not in norm['actions']
            node.violation = node.name not in norm['actions'] and node.type == 'ACT'

    # For OR nodes, the node violates if all children violate
    if hasattr(node, 'type') and node.type == 'OR':
        node.violation = all(child.violation for child in node.children)
    # For SEQ/AND nodes, the node violates if any child violates
    elif hasattr(node, 'type') and node.type in ['SEQ', 'AND']:
        node.violation = any(child.violation for child in node.children)

def export_tree_to_png(root, output_file):
    """
    Exports the tree to a PNG file with node properties.

    Parameters:
    root (Node): The root node of the tree
    output_file (string): The path to the output file
    """
    DotExporter(root, 
                nodeattrfunc=lambda node: f'label="{node.name}\nViolation: {node.violation}"'
               ).to_picture(output_file)

def make_decision(json_tree, norm, goal, beliefs, preferences, output_dir=""):
    """
    Main function to determine the best execution trace for the agent.

    Parameters:
    json_tree (json object): The goal tree 
    norm (dict): The norm
    goal (list): The goal of the agent: a set of beliefs (strings) of the agent that must be true at the end of the execution of the trace.
    beliefs (list): A set of strings representing the initial beliefs of the agents.
    preferences (list): A pair describing the preference of the end-user.
    output_dir (string): The directory to save the output image

    Returns:
    tree (Node): The root node of the tree
    chosen_trace (list): A list of strings representing the execution trace chosen by the agent.
    valid_traces (list): A list of all valid traces
    valid_costs (list): A list of the cost of each valid trace
    """
    # Build the tree from the JSON object
    root = build_tree(json_tree)

    # Annotate the tree based on the given norm
    annotate_tree(root, norm)

    # Generate all possible traces of the given tree, and calculate the cost of each trace
    traces, costs = generate_traces(root, calc_cost=True)

    if print_mode:
        print("costs: ", costs)
        print(f"Generated {len(traces)} traces:")
        for trace in traces:
            print(trace)

    # Filter traces that violate norms and keep their respective costs
    valid_traces = []
    valid_costs = []
    
    for trace, cost in zip(traces, costs):
        valid = True
        has_all_goals = [False for goal_belief in goal]
        agent_beliefs = beliefs.copy()

        for node_name in trace:
            node = find(root, lambda node: node.name == node_name)

            # Check if node found
            if not node:
                continue

            # Check if the node violates any norms
            if node.violation:
                valid = False
                if print_mode:
                    print(f"Trace violates norm: {trace}")
                break  

            # Check if the node violates any preconditions
            if (node and hasattr(node, 'pre') and any(pre not in agent_beliefs for pre in node.pre)):
                valid = False
                if print_mode:
                    print(f"Trace violates beliefs: {trace}")
                    print(f"Current Agent Beliefs: {agent_beliefs}")
                    print(f"Node pre: {node.pre}")
                break

            # Update agent beliefs given the execution of the current node
            if node and hasattr(node, 'post'):
                agent_beliefs.extend(node.post)

                # Check if all goals are achieved
                for i, goal_belief in enumerate(goal):
                    if hasattr(node, 'post') and goal_belief in node.post:
                        has_all_goals[i] = True

        # If the trace is valid and all goals are achieved, add it to the valid traces
        if valid and all(has_all_goals):            
            valid_traces.append(trace)
            valid_costs.append(cost)

    # Sort traces based on user preferences
    if preferences and len(preferences) == 2:
        indices = preferences[1]
        sorted_traces_and_costs = sorted(zip(valid_traces, valid_costs), key=lambda x: tuple(x[1][i] for i in indices))
        valid_traces, valid_costs = zip(*sorted_traces_and_costs) if sorted_traces_and_costs else ([], [])

    # Return the best trace
    chosen_trace = valid_traces[0] if valid_traces else []
    if print_mode:
        print(f"Best trace: {chosen_trace}")
    
    return root, chosen_trace, valid_traces, valid_costs


def create_explanation(key="", node_name=None, value=[]):
    """
    Creates an explanation for a given key, node name, and value.

    Parameters:
    key (string): The key of the explanation
    node_name (string): The name of the node
    value (list): The value of the explanation

    Returns:
    list: A list representing the explanation
    """

    if node_name is None:
        return [key] + value
    
    return [key, node_name] + value


def add_explanation(explanations, key="", node_name=None, value=[]):
    """
    Adds an explanation to the list of explanations.

    Parameters:
    explanations (list): The list of explanations
    key (string): The key of the explanation
    node_name (string): The name of the node
    value (list): The value of the explanation
    """
    explanations.append(create_explanation(key, node_name, value))

def get_cost_of_node(traces, costs, node):
    """
    Calculate the cost of a given node.

    Parameters:
    traces (list): A list of all possible traces
    costs (list): A list of the cost of each trace
    node (Node): The node for which to calculate the cost

    Returns:
    list: The cost of the node
    """
    if not traces or not costs or not node:
        return []
    
    if hasattr(node, 'costs'):
        return node.costs

    # Find the index of the node in the traces
    node_index = next((i for i, trace in enumerate(traces) if node.name in trace), None)

    # If the node is not found in the traces, return an empty list
    if node_index is None:
        return []
    
    # Return the cost of the node
    return costs[node_index]

def add_linked_node_explanations(explanations, node, root):
     if hasattr(node, 'link') and node.link:
        for dest_node_name in node.link:
            linked_node = find_node(root, dest_node_name)
            if linked_node:
                add_explanation(explanations, key='L', node_name=node.name, value=['->', dest_node_name])
            if hasattr(linked_node, 'link') and node.link:
                add_linked_node_explanations(explanations, linked_node, root)



def generate_formal_explanations(json_tree, norm, goal, beliefs, preferences, action_to_explain, output_dir=""):
    """
    Explain why a certain action was executed as part of a selected execution trace

    Parameters:
    json_tree (json object): The goal tree 
    norm (dict): The norm
    goal (list): The goal of the agent: a set of beliefs (strings) of the agent that must be true at the end of the execution of the trace.
    beliefs (list): A set of strings representing the initial beliefs of the agents.
    preferences (list): A pair describing the preference of the end-user.
    action_to_explain (string): The name of the action to explain.
    output_dir (string): The directory to save the output image

    Returns:
    output (list): A list of strings representing the execution trace chosen by the agent.
    chosen_trace (list): A list of strings representing the execution trace chosen by the agent.
    """

    explanations = []

    # Use part 3 code to get selected trace - Get the root node, the chosen trace, and the valid traces
    root, chosen_trace, valid_traces, valid_costs = make_decision(json_tree, norm, goal, beliefs, preferences, output_dir)


    print(f"Chosen trace: {chosen_trace}")
    if not chosen_trace or len(chosen_trace) == 0 or action_to_explain not in chosen_trace:
        print(f"Action to explain: {action_to_explain} not in trace")
        # If the action is not in the trace, return an empty list
        return [], chosen_trace
    else:
        print(f"Action to explain: {action_to_explain} in trace")


    # Get target node to explain
    target_node = find_node(root, action_to_explain)
    if not target_node:
        return [], chosen_trace

    if hasattr(target_node, 'ancestors'):
        ancestor_names = [ancestor.name for ancestor in target_node.ancestors]
        print(f"Ancestor names: {ancestor_names}")

    """
    Starting generating the explanation, according to the pdf it should contain a list of explanatory factors as defined below.
    The list should be obtained by traversing the tree in pre-order.
    """    
    agent_beliefs = beliefs.copy()
    for node in PreOrderIter(root):
        current_node_name = node.name
        current_node_type = None
        if hasattr(node, 'type'):
            current_node_type = node.type

        if current_node_name in chosen_trace:
            """ 
            (a) Pre-conditions of an action (denoted with a “P"). Requested format:
                ['P', action name,
                list of preconditions of the actions (including A) that were satisfied and that
                made the execution of action A that is being explained possible]
                Example: ['P', 'getOwnCard', ['ownCard']]
                Note: No "P" factor should be included in the list if an action has no preconditions.
            """
            if hasattr(node, 'pre') and node.name in chosen_trace and hasattr(node, 'type') and node.type in ['ACT']:
                # Add preconditions of cuurent action to the global preconditions list
                add_explanation(explanations, key='P', node_name=node.name, value=[node.pre.copy()])
            
            # Update agent beliefs given the execution of the current node
            if hasattr(node, 'post'):
                agent_beliefs.extend(node.post)

            # Handle OR nodes (b, c, d, e explanations [C, V, N, F])
            if current_node_type == 'OR':
                chosen_child = next(child for child in node.children if child.name in chosen_trace)
                chosen_child_name = chosen_child.name
                for child in node.children:
                    child_name = child.name
                    if child_name in chosen_trace:
                        """
                        (b) A condition of a choice (“C"). Requested format:
                            ['C', name of alternative that was chosen for an OR node,
                            list of preconditions of the alternative that were satisfied and that
                            made the choice possible]
                            Example: ['C', 'getKitchenCoffee', ['staffCardAvailable']]
                            Note: getKitchenCoffee is one of the alternatives of the getCoffee OR node
                        """
                        add_explanation(explanations, key='C', node_name=child_name, value=[chosen_child.pre.copy()])
                    else:
                        """
                        An explanation for each alternative not selected, either via a ”N” factor, a
                        ”V” factor, or a ”F” factor explanation, depending on the reason. Note:
                        if, for one alternative not selected, multiple reasons are true, only report
                        the first factor, considering the order above (i.e., a ”V” factor only if no
                        ”N” factor is relevant, and a ”F” factor only if no ”N” nor ”V” factors are
                        releant).
                        """

                        """
                        (d) A norm ("N"). Requested format:
                            ['N', name of an alternative of an OR node that was NOT chosen because
                            it violates (possibly through its children) a norm,
                            the norm that is violated]
                            Example: ['N', 'getShopCoffee', 'P(payShop)']
                        """
                        if hasattr(child, "violation") and child.violation:
                            norm_string = f"{norm['type']}({', '.join(norm['actions'])})"
                            add_explanation(explanations, key='N', node_name=child.name, value=[norm_string])
                            continue
                        """
                        (c) A value statement (“V"). Requested format:
                            ['V', name of an alternative that was chosen for an OR node,
                            list of costs for that alternative,
                            '>',
                            name of another alternative of an OR node that was NOT chosen,
                            list of costs for that alternative]
                            Example:
                            ['V', 'getKitchenCoffee', [5.0, 0.0, 3.0],
                            '>',
                            'getAnnOfficeCoffee', [2.0, 0.0, 6.0]]
                        """   
                        unsatisfied_preconditions = []
                        if hasattr(child, 'pre'):
                            unsatisfied_preconditions = [pre for pre in child.pre if pre not in agent_beliefs]             

                        if len(unsatisfied_preconditions) == 0:
                            chosen_child_cost = get_cost_of_node(valid_traces, valid_costs, chosen_child)                            
                            child_cost = get_cost_of_node(valid_traces, valid_costs, child)
                            child_cost_formatted = child_cost.tolist() if isinstance(child_cost, np.ndarray) else child_cost
                            chosen_child_cost_formatted = chosen_child_cost.tolist() if isinstance(chosen_child_cost, np.ndarray) else chosen_child_cost
                            add_explanation(explanations, key='V', 
                                            node_name=chosen_child_name, 
                                            value=[chosen_child_cost_formatted, '>',child_name, child_cost_formatted])
                            continue
                        

                        """
                        (e) A failed condition of a choice ("F"). Requested format:
                            ['F', name of an alternative of an OR node that was NOT chosen because
                            (some of) its pre-conditions were not satisfied,
                            list of preconditions of the alternative that were NOT
                            satisfied and made the choice not possible]
                            Example: ['F', 'getKitchenCoffee', ['staffCardAvailable']]
                        """                        
                        add_explanation(explanations, key='F', node_name=child_name, value=[unsatisfied_preconditions])
                        continue
            
            """ (g) A goal ("D"). Requested format:
                ['D', name of the goal]
                Example: ['D', 'getKitchenCoffee']
            """
            if current_node_type in ['SEQ', 'AND', 'OR'] and ancestor_names and len(ancestor_names) > 0 and current_node_name in ancestor_names:
                add_explanation(explanations, key='D', value=[current_node_name])

        # Check if the current node is the action to explain
        if current_node_name == action_to_explain:
            """
            (f) A link (“L"). Requested format:
                ['L', name of the node, '->', name of the linked node]
                Example: ['L', 'payShop', '->', 'getCoffeeShop']
                Note: a node a links to another node b if a's attribute "link" contains the
                name of b. Furthermore, if the linked node b also has a link to another node c,
                then the explanation should also include such a link (and all the links forming
                a chain starting from a) to the explanation, i.e., for each link in the chain an
                explanation in the requested format above should included in the list.
            """
            add_linked_node_explanations(explanations, node, root)
            # Stop the traversal if the action to explain is reached
            break

    

    """ (h) The user preference ("U"). Requested format:
        ['U' the pair given in input as user preference]
        Example: ['U', [['quality', 'price', 'time'], [1, 2, 0]]]
    """
    if preferences and len(preferences) == 2:
        add_explanation(explanations, key='U', value=[preferences])


    return explanations, chosen_trace



"""
Generate naive baseline approach of if-else rules to generate natural language explanations
"""

def handle_condition_explanation(action, formal_explention):
    conditions = ', '.join(formal_explention[2])
    return f"The agent was able to perform the action '{action}' as all its preconditions ({conditions}) were successfully met."

def handle_value_comparison_explanation(action1, values1, operator, action2, values2, preferences_labels, preferences_values, preference_labels_formatted):
    value1_formatted = ', '.join([f"{label}: {value}" for label, value in zip(preferences_labels, values1)])
    value2_formatted = ', '.join([f"{label}: {value}" for label, value in zip(preferences_labels, values2)])
    
    if values1 == values2:
        return f"Both actions have the same values for all factors ({value1_formatted}), so the agent randomly chose '{action1}' over '{action2}."
    
    explanation = f"The agent chose '{action1}' over '{action2}' based on the user's preferences to prioritize {preference_labels_formatted}"

    # Compare the values succinctly
    explanation += f" and given that '{action1}' has these values: " + value1_formatted
    explanation += f", while '{action2}' has: " + value2_formatted

    # Find the decisive factor
    for index in preferences_values:
        preference_label = preferences_labels[index]
        value1 = values1[index]
        value2 = values2[index]

        if value1 != value2:
            explanation += f". The decisive factor was the {preference_label}, as '{action2}' had a higher value of {value2} compared to '{action1}' with {value1}. Lower values are more desirable, so '{action1}' was chosen."
            break

    # If all values are equal
    if not any(value1 != value2 for value1, value2 in zip(values1, values2)):
        explanation += f". Since all factors were equally matched, the agent randomly chose the first action ({action1})."

    return explanation

def handle_norm_violation_explanation(action, formal_explention):
    norm = formal_explention[2]
    norm_type = norm[0]
    norm_action = norm[2:-1]
    
    if norm_type == "P":  # Prohibition Norm
        if norm_action == action:
            return f"The action '{action}' is not allowed because it violates the prohibition norm, which states that this action cannot be executed."
        
        return f"The action '{action}' is not allowed because it leads to the execution of a prohibited action. The prohibition norm specifies that the following action must not be performed: {norm_action}."
    
    elif norm_type == "O":  # Obligation Norm
        return f"The action '{action}' is not allowed because it violates the obligation norm, which specifies that only the following action(s) are permitted: {norm_action}."

def handle_precondition_explanation(action, formal_explention):
    preconditions = formal_explention[2]
    if len(preconditions) > 1:
        preconditions_formatted = ', '.join(preconditions[:-1]) + ', and ' + preconditions[-1]
        return f"The agent could not perform '{action}' because the obligatory preconditions: {preconditions_formatted} were not met."
    else:
        preconditions_formatted = preconditions[0]
        return f"The agent could not perform '{action}' because the obligatory precondition: {preconditions_formatted} was not met."

def handle_decision_explanation(action):
    return f"The agent chose to perform '{action}' as it is a necessary step to achieve the goal action."

def handle_link_explanation(action, formal_explention):
    linked_action = formal_explention[3]
    return f"The action '{action}' is linked to the action '{linked_action}', and as such, it was executed as a necessary step to achieve the goal action."

def handle_utility_function_explanation(preferences):
    preferences_labels = preferences[0]
    preferences_values = preferences[1]
    # Sort preferences_labels according to preferences_values
    preference_labels_sorted = [preferences_labels[i] for i in preferences_values]
    preference_labels_formatted = format_list_to_string(preference_labels_sorted)
    return f"The agent's preferences, in descending order of importance, are: {preference_labels_formatted}."

def handle_failure_explanation(action, formal_explention):
    """
    (e) A failed condition of a choice ("F"). Requested format:
        ['F', name of an alternative of an OR node that was NOT chosen because
        (some of) its pre-conditions were not satisfied,
        list of preconditions of the alternative that were NOT
        satisfied and made the choice not possible]
        Example: ['F', 'getKitchenCoffee', ['staffCardAvailable']]
    """     
    # action = formal_explention[1]
    preconditions = formal_explention[2]
    preconditions_labels_formatted = format_list_to_string(preconditions)
    
    return f"The agent cound not executed '{action}' because of the preconditions for it, which are {preconditions_labels_formatted}, were not met."

def generate_natural_explentions(formal_explention, preferences):
    """
    Generate naive baseline approach of if-else rules to generate natural language explanations
    :param formal_explention: a single formal explentions (output of part1-4)
    :param preferences: user preferences
    :return: natural language explention
    """
    explanation_type = formal_explention[0]
    action = formal_explention[1]
    preferences_labels = preferences[0]
    preferences_values = preferences[1]
    # Sort preferences_labels according to preferences_values
    preference_labels_sorted = [preferences_labels[i] for i in preferences_values]
    preference_labels_formatted = format_list_to_string(preference_labels_sorted)

    if explanation_type == 'C':  # Condition
        return handle_condition_explanation(action, formal_explention)
    elif explanation_type == 'V':  # Value Comparison
        action1, values1, operator, action2, values2 = formal_explention[1:]
        return handle_value_comparison_explanation(action1, values1, operator, action2, values2, preferences_labels, preferences_values, preference_labels_formatted)
    elif explanation_type == 'N':  # Norm Violation
        return handle_norm_violation_explanation(action, formal_explention)
    elif explanation_type == 'P':  # Precondition
        return handle_precondition_explanation(action, formal_explention)
    elif explanation_type == 'D':  # Decision
        return handle_decision_explanation(action)
    elif explanation_type == 'L':  # Link
        return handle_link_explanation(action, formal_explention)
    elif explanation_type == 'U':  # Utility Function
        return handle_utility_function_explanation(preferences)
    elif explanation_type == 'F':
        return handle_failure_explanation(action, formal_explention)
    
    return "Unknown explanation type."

def format_list_to_string(lst):
    """
    Format a list of strings into a single string with commas and 'and' before the last element.
    :param lst: The list of strings to format.
    :return: The formatted string.
    """

    if len(lst) > 1:
        return ', '.join(lst[:-1]) + ', and ' + lst[-1]
    return lst[0]

def generate_restriction_description(norm, goal, beliefs, preferences, action_to_explain):
    """
    Generate a natural language description of the restrictions placed on the agent.
    
    Parameters:
    norm (dict): The norm restricting or obligating actions.
    goal (list): The desired outcome, represented as a set of beliefs that must be true by the end of execution.
    beliefs (list): The agent's initial knowledge.
    preferences (list): A pair describing the end-user's priority order.
    action_to_explain (string): The action being analyzed.
    
    :return: A natural language description of the restrictions.
    """

    restrictions = []

    # Describe norm
    if norm:
        norm_type = norm.get("type", "Unknown")
        norm_actions = norm.get("actions", [])

        if norm_actions:
            norm_action_str = format_list_to_string(norm_actions)
            if norm_type == "P":
                restrictions.append(f"Restricted actions: {norm_action_str}.")
            elif norm_type == "O":
                restrictions.append(f"Required actions: {norm_action_str}.")

    # Describe initial beliefs
    if beliefs:
        restrictions.append(f"Starting beliefs: {format_list_to_string(beliefs)}.")

    # Describe goal
    if goal:
        restrictions.append(f"Goal: {format_list_to_string(goal)}.")

    # Describe preferences
    preferences_description = handle_utility_function_explanation(preferences)
    restrictions.append(preferences_description)

    # Combine everything into a natural flow
    restriction_sentence = " ".join(restrictions)
    return f"To explain the action '{action_to_explain}', consider the following context: {restriction_sentence}"


def generate_naive_baseline(formal_explentions, chosen_trace, norm, goal, beliefs, preferences, action_to_explain):
    """
    Generate naive baseline approach of if-else rules to generate natural language explanations
    
    Parameters:
    formal_explentions (list): list of formal explentions (output of part1-4)
    chosen_trace (list): chosen trace
    norm (dict): The norm
    goal (list): The goal of the agent: a set of beliefs (strings) of the agent that must be true at the end of the execution of the trace.
    beliefs (list): A set of strings representing the initial beliefs of the agents.
    preferences (list): A pair describing the preference of the end-user.
    action_to_explain (string): The name of the action to explain.

    :return: list of natural language explentions
    """
    natural_explentions = []
    # Format the chosen trace for display
    chosen_trace_formatted = ""
    if chosen_trace is not None and len(chosen_trace) > 0:
       chosen_trace_formatted = format_list_to_string(chosen_trace)

    restriction_description = generate_restriction_description(norm, goal, beliefs, preferences, action_to_explain)
    natural_explentions.append(restriction_description)

    if formal_explentions is None or len(formal_explentions) == 0:
        if chosen_trace:
            natural_explentions.append(
                f"The action '{action_to_explain}' was not executed in the chosen trace ({chosen_trace_formatted})."
            )

        if norm and norm.get("type") == "P" and action_to_explain in norm.get("actions", []):
            natural_explentions.append("This action is restricted by the norm and therefore could not be executed.")

        return natural_explentions

    # Generate natural language explanations for each formal explanation
    for explention in formal_explentions:
        natural_explentions.append(generate_natural_explentions(explention, preferences))

    return natural_explentions




### Changed pipeline a bit, added preorder_traversal list to follow the order in which the agent decides what to do;
### function generate_comprehensive_explanation generates response from mistral https://ollama.com/library/mistral

In [2]:
import requests

# Define the API endpoints
api_base = "http://localhost:11434/api/v1"
embedding_api_base = "http://localhost:11434/api/v1"

# Example request to the LLM model
llm_payload = {
    "input": "Translate formal explanation to natural language: C: getKitchenCoffee requires [staffCardAvailable]"
}
response = requests.post(f"{api_base}/generate", json=llm_payload)

# Debugging: Print response status code and content
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.text}")

try:
    print(response.json())
except requests.exceptions.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

# Example request to the embedding model
embedding_payload = {
    "input": "This is a sample text to embed."
}
response = requests.post(f"{embedding_api_base}/embed", json=embedding_payload)

# Debugging: Print response status code and content
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.text}")

try:
    print(response.json())
except requests.exceptions.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

Status Code: 404
Response Content: 404 page not found
JSONDecodeError: Extra data: line 1 column 5 (char 4)
Status Code: 404
Response Content: 404 page not found
JSONDecodeError: Extra data: line 1 column 5 (char 4)


In [ ]:
import ollama


from anytree import PreOrderIter

def preorder_traversal(root):
    """
    Performs a preorder traversal of a tree and returns a list of node names.
    
    :param root: The root node of the tree.
    :return: A list of node names in preorder traversal order.
    """
    preorder_traversal_nodes = []
    
    for node in PreOrderIter(root):
        current_node_name = node.name
        print(current_node_name)
        preorder_traversal_nodes.append(current_node_name)
    
    return preorder_traversal_nodes

def generate_natural_language_explanation(naive_explanation, formal_explanations, tree_traversal_order, 
                                     norm, beliefs, goal, preferences, action_to_explain, chosen_trace=None):
    
    # Format key context elements in plain English
    norm_str = norm if norm else "None"
    beliefs_str = ", ".join(beliefs) if beliefs else "None"
    goal_str = goal if goal else "None"
    pref_description = f"{preferences[0]} with costs {preferences[1]}"
    chosen_trace_str = ", ".join(chosen_trace) if chosen_trace else "None"
    
    prompt = f"""
    You are an AI agent explaining your decision-making process to a human user.
    
    Generate a natural, conversational explanation about why you performed a specific action while achieving a goal.
    Do no use technical terms or jargon. Explain your reasoning in a way that a non-expert can understand.
    Do not be overly verbose. Keep the explanation concise and to the point.
    Do not generate exentric or creative explanations. Stick to the facts and reasoning.
    Include at the begining all information of restrictions, beliefs, goals, preferences, and actions, then the selected trace and only then start explining the reasoning.
    If agent faild with explaning the action or there was no selected trace state that at the begining.
    
    CONTEXT:
    - Action I need to explain: {action_to_explain}
    - Norm/restriction I know: {norm_str}
    - My initial beliefs: {beliefs_str}
    - Goal I was given: {goal_str}
    - User's preferences: {pref_description}
    - Actions I performed: {chosen_trace_str}
    
    HOW I EXPLORED OPTIONS (reasoning tree traversal):
    {tree_traversal_order}
    
    TECHNICAL EXPLANATION (needs conversion to natural language):
    {naive_explanation}
    
    DETAILED REASONING CODES:
    {formal_explanations}
    
    YOUR TASK:
    Create a natural, conversational explanation that:
    1. Follows the exact reasoning path I took when making decisions
    2. Starts by acknowledging the user's goal and preferences
    3. Explains each decision point in order, showing:
       - How I started with the main goal
       - How I considered different options
       - Why I chose certain actions and rejected others
       - How preconditions, norms, and preferences affected my choices
    4. Translates each reasoning code into natural language:
       - P codes → explain what conditions made an action possible
       - C codes → explain what allowed me to make a choice
       - F codes → explain why some options weren't possible
       - N codes → explain which norms prevented certain actions
       - V codes → explain why I preferred one option over another
       - L codes → explain how actions are connected or dependent
       - D codes → explain how actions helped achieve the goal
       - U codes → explain how user preferences guided my decisions
    
    IMPORTANT:
    - Write in first person ("I") as if I'm the agent speaking directly to the user
    - Use simple, conversational language (no technical terms)
    - Create a flowing narrative that shows how one decision led to the next
    - Specifically explain why I performed {action_to_explain}
    - Keep the explanation concise but complete
    - Write as a flowing paragraph, not a list or bullet points


    Deeper context on how the formal explanations were generated:
    (a) Pre-conditions of an action (denoted with a “P"). Requested format:
                ['P', action name,
                list of preconditions of the actions (including A) that were satisfied and that
                made the execution of action A that is being explained possible]
                Example: ['P', 'getOwnCard', ['ownCard']]
                Note: No "P" factor should be included in the list if an action has no preconditions.
    
    (b) A condition of a choice (“C"). Requested format:
                            ['C', name of alternative that was chosen for an OR node,
                            list of preconditions of the alternative that were satisfied and that
                            made the choice possible]
                            Example: ['C', 'getKitchenCoffee', ['staffCardAvailable']]
                            Note: getKitchenCoffee is one of the alternatives of the getCoffee OR node
    
    (c) A value statement (“V"). Requested format:
                            ['V', name of an alternative that was chosen for an OR node,
                            list of costs for that alternative,
                            '>',
                            name of another alternative of an OR node that was NOT chosen,
                            list of costs for that alternative]
                            Example:
                            ['V', 'getKitchenCoffee', [5.0, 0.0, 3.0],
                            '>',
                            'getAnnOfficeCoffee', [2.0, 0.0, 6.0]]
    
    (d) A norm ("N"). Requested format:
                            ['N', name of an alternative of an OR node that was NOT chosen because
                            it violates (possibly through its children) a norm,
                            the norm that is violated]
                            Example: ['N', 'getShopCoffee', 'P(payShop)']
    
    (e) A failed condition of a choice ("F"). Requested format:
                            ['F', name of an alternative of an OR node that was NOT chosen because
                            (some of) its pre-conditions were not satisfied,
                            list of preconditions of the alternative that were NOT
                            satisfied and made the choice not possible]
                            Example: ['F', 'getKitchenCoffee', ['staffCardAvailable']]
    
    
    (f) A link (“L"). Requested format:
                ['L', name of the node, '->', name of the linked node]
                Example: ['L', 'payShop', '->', 'getCoffeeShop']
                Note: a node a links to another node b if a's attribute "link" contains the
                name of b. Furthermore, if the linked node b also has a link to another node c,
                then the explanation should also include such a link (and all the links forming
                a chain starting from a) to the explanation, i.e., for each link in the chain an
                explanation in the requested format above should included in the list.
    
    (g) A goal ("D"). Requested format:
                ['D', name of the goal]
                Example: ['D', 'getKitchenCoffee']
    
    (h) The user preference ("U"). Requested format:
        ['U' the pair given in input as user preference]
        Example: ['U', [['quality', 'price', 'time'], [1, 2, 0]]]
    """
    
    # Send the prompt to your LLM
    api_base = "http://localhost:11434"
    messages = [
        {
            'role': 'user',
            'content': prompt
        }
    ]
    
    # response = requests.post(f"{api_base}/chat", json={"model": "mistral", "messages": messages})
    response = ollama.chat(model='mistral', messages=messages)
    
    try:
        return response['message']['content']
    except requests.exceptions.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        return None


def preorder_traversal(root):
    """
    Performs a preorder traversal of a tree and returns a list of node names.
    
    :param root: The root node of the tree.
    :return: A list of node names in preorder traversal order.
    """
    preorder_traversal_nodes = []
    
    for node in PreOrderIter(root):
        current_node_name = node.name
        print(current_node_name)
        preorder_traversal_nodes.append(current_node_name)
    
    return preorder_traversal_nodes




#### Integrated here llm explanations and save them into explanations/mistral

In [11]:
def generate_explanations_for_inputs(test_cases):
    """
    Generate explanations for the inputs test cases
    :param test_cases: dictionary containing the test cases
    :return: dictionary containing the explanations for each test case
    """
    
    # in .ipynb current directory is extracted like this
    current_dir = os.getcwd()

    # current_dir = os.path.dirname(__file__)

    baseline_explanations_dir = os.path.join(current_dir, "explanations", "baseline")
    os.makedirs(baseline_explanations_dir, exist_ok=True)


    llm_explanations_dir = os.path.join(current_dir, "explanations", "mistral")
    
    baseline_explanations = {}
    llm_explanations = {}
    for index, row in test_cases.iterrows():
        # Read the JSON file into a dictionary
        with open(f'{current_dir}/{row["name_json_tree_file"]}', 'r') as file:
            json_tree = json.load(file)
        # Generate formal explanations
        formal_explentions, chosen_trace = generate_formal_explanations(json_tree=json_tree, norm=row["norm"], beliefs=row["beliefs"], goal=row["goal"], preferences=row["preferences"], action_to_explain=row["action_to_explain"])
        baseline_explanations[index] = generate_naive_baseline(formal_explentions=formal_explentions, chosen_trace=chosen_trace, norm=row["norm"], beliefs=row["beliefs"], goal=row["goal"], preferences=row["preferences"], action_to_explain=row["action_to_explain"])



        root, chosen_trace, valid_traces, valid_costs = make_decision(json_tree, norm=row["norm"], goal=row["goal"], beliefs=row["beliefs"], preferences=row["preferences"], output_dir='/')
        tree_traversal_order = preorder_traversal(root=root)

        # print(f"naive_explanation {baseline_explanations[index]}, formal_explanations {formal_explentions}, norm {row['norm']}, beliefs {row['beliefs']}, preferences {row['preferences']}")


        llm_explanations[index] = generate_natural_language_explanation(
                        naive_explanation=baseline_explanations[index],
                        formal_explanations=formal_explentions,
                        tree_traversal_order=tree_traversal_order,
                        norm=row["norm"],
                        beliefs=row["beliefs"],
                        goal=row["goal"],
                        preferences=row["preferences"],
                        action_to_explain=row["action_to_explain"],
                        chosen_trace=chosen_trace
                        )

        # Save each explanation to a separate text file
        baseline_explanation_file = os.path.join(baseline_explanations_dir, f"{index}.txt")
        with open(baseline_explanation_file, 'w') as f:
            for explanation in baseline_explanations[index]:
                f.write(explanation + '\n')


        os.makedirs(llm_explanations_dir, exist_ok=True)  

        llm_explanation_file = os.path.join(llm_explanations_dir, f"{index}.txt")

        print(f"directory ---> {llm_explanation_file}")
        
        # with open(llm_explanation_file, 'w') as f:
        #     for explanation in llm_explanations[index]:
        #         f.write(explanation + '\n')

        # Loop through the dictionary and write explanations for each test case
        with open(llm_explanation_file, 'w') as f:
            for explanation in llm_explanations[index]:
                # Write the explanation for this test case
                f.write(explanation)   # Add an extra newline for separation

        
    
    return baseline_explanations, llm_explanations

In [12]:
test_cases = [
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["payShop"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [1, 2, 0]],
        "action_to_explain": "payShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {},
        "beliefs": ["staffCardAvailable", "ownCard"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "getCoffeeKitchen"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {},
        "beliefs": ["haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "getCoffeeShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["gotoAnnOffice"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "payShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["payShop"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [1, 2, 0]],
        "action_to_explain": "gotoKitchen"
    }
]

df_test_cases = pd.DataFrame(test_cases, index=[f"test_case_{i+1}" for i in range(len(test_cases))])

In [13]:
baseline_explanations, llm_explanations = generate_explanations_for_inputs(df_test_cases)

Chosen trace: ['getCoffee', 'getKitchenCoffee', 'getStaffCard', 'getOwnCard', 'gotoKitchen', 'getCoffeeKitchen']
Action to explain: payShop not in trace
getCoffee
getKitchenCoffee
getStaffCard
getOwnCard
getOthersCard
gotoKitchen
getCoffeeKitchen
getAnnOfficeCoffee
gotoAnnOffice
getPod
getCoffeeAnnOffice
getShopCoffee
gotoShop
payShop
getCoffeeShop
directory ---> c:\Users\bar24\OneDrive - Universiteit Utrecht\Documents\School\UU Data Sceince MSc\1st Year\Period 3\Explainable AI - INFOMXAI\Assignments\Project2\ExplainableAI-Project2\Part2\explanations\mistral\test_case_1.txt
Chosen trace: ['getCoffee', 'getKitchenCoffee', 'getStaffCard', 'getOwnCard', 'gotoKitchen', 'getCoffeeKitchen']
Action to explain: getCoffeeKitchen in trace
Ancestor names: ['getCoffee', 'getKitchenCoffee']
getCoffee
getKitchenCoffee
getStaffCard
getOwnCard
getOthersCard
gotoKitchen
getCoffeeKitchen
getAnnOfficeCoffee
gotoAnnOffice
getPod
getCoffeeAnnOffice
getShopCoffee
gotoShop
payShop
getCoffeeShop
directory ---

In [ ]:
import requests

# Define the API endpoints
api_base = "http://localhost:11434"

# Example request to the LLM model
llm_payload = {
    "input": "Translate formal explanation to natural language: C: getKitchenCoffee requires [staffCardAvailable]"
}
response = requests.post(f"{api_base}", json=llm_payload)

# Debugging: Print response status code and content
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.text}")

try:
    response_json = response.json()
    print(response_json)
except requests.exceptions.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

# Example request to the embedding model
embedding_payload = {
    "input": "This is a sample text to embed."
}
response = requests.post(f"{api_base}/embed", json=embedding_payload)

# Debugging: Print response status code and content
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.text}")

try:
    response_json = response.json()
    print(response_json)
except requests.exceptions.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")